In [1]:
import os
import cv2
import random
import shutil
import xml.dom.minidom
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans

### move jpgs with xmls/txts out to separate directory

In [ ]:
def scan_files(directory, prefix=None, postfix=None):
    files_list = []
    for root, sub_dirs, files in os.walk(directory):
        for special_file in files:
            if postfix:
                if special_file.endswith(postfix):
                    files_list.append(os.path.join(root, special_file))
            elif prefix:
                if special_file.startswith(prefix):
                    files_list.append(os.path.join(root, special_file))
            else:
                files_list.append(os.path.join(root, special_file))
    return files_list

def copy_jpg_and_labels(from_path, to_path):
    image_names = scan_files(from_path, postfix=".jpg")
    for image_name in image_names:
        xml_name = os.path.splitext(image_name)[0] + ".xml"
        txt_name = os.path.splitext(image_name)[0] + ".txt"
        if os.path.isfile(xml_name):
            shutil.move(image_name, to_path)
            shutil.move(xml_name, to_path)
        elif os.path.isfile(txt_name):
            shutil.move(image_name, to_path)
            shutil.move(txt_name, to_path)


orig_path = "../../yolo_sc_20181128/SC_orig"
tmp_path = "../../yolo_sc_20181128/SC_tmp"

copy_jpg_and_labels(orig_path, tmp_path)

### find proper training input image size (608x608)

In [2]:
# collect all sizes of images
tmp_path = "../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done"
image_names = [os.path.join(tmp_path, f) for f in os.listdir(tmp_path) if f.endswith(".jpg")]
print("# of jpgs", len(image_names))
image_sizes = []
for image_name in image_names:
    with Image.open(image_name) as image:
        image_sizes.append(image.size)

# of jpgs 13073


In [3]:
# kmeans clustering
X = np.array(image_sizes)
kmeans = KMeans(n_clusters=9, random_state=2018).fit(X)
print(X.shape, kmeans)

(13073, 2) KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=9, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=2018, tol=0.0001, verbose=0)


In [4]:
centers = []
for center in kmeans.cluster_centers_:
    print(center)
    centers.append(center)

[272.67065868 155.16167665]
[127.30975769 108.91421087]
[161.00606061 250.90909091]
[99.98157988 96.98476798]
[138.50323974 139.89956803]
[103.2854902  130.61568627]
[170.38482385 113.56278229]
[112.72  176.424]
[190.61408451 179.88169014]


In [5]:
total = len(image_sizes)
large = len([image_size for image_size in image_sizes if image_size[0] > 608 or image_size[1] > 608])
print(total, large)

13073 0


### image padding and resizing

In [6]:
def read_labels_from_xml(xml_name):
    DOMTree = xml.dom.minidom.parse(xml_name)
    collection = DOMTree.documentElement
    filename = collection.getElementsByTagName("filename")
    objects = collection.getElementsByTagName("object")

    w = collection.getElementsByTagName("width")[0]
    w_val = int(w.firstChild.nodeValue)
    h = collection.getElementsByTagName("height")[0]
    h_val = int(h.firstChild.nodeValue)
    w.firstChild.replaceWholeText(str(h_val))
    h.firstChild.replaceWholeText(str(w_val))

    labels = {'w':w_val, 'h':h_val, 'boxes':[]}
    
    for object in objects:
        xmin = object.getElementsByTagName("xmin")[0]
        xmin_val = int(xmin.firstChild.nodeValue)
        xmax = object.getElementsByTagName("xmax")[0]
        xmax_val = int(xmax.firstChild.nodeValue)
        ymin = object.getElementsByTagName("ymin")[0]
        ymin_val = int(ymin.firstChild.nodeValue)
        ymax = object.getElementsByTagName("ymax")[0]
        ymax_val = int(ymax.firstChild.nodeValue)
        labels["boxes"].append([xmin_val, ymin_val, xmax_val, ymax_val])
        
    return labels


def read_labels_from_txt(txt_name):
    jpg_name = os.path.splitext(txt_name)[0] + ".jpg"
    with Image.open(jpg_name) as img:
        w, h = img.size
        
    labels = {'w':w, 'h':h, 'boxes':[]}
    with open(txt_name, 'r') as f:
        for line in f.readlines():
            tokens = line.strip().split()
            xc, yc = w*float(tokens[1]), h*float(tokens[2])
            w_, h_ = w*float(tokens[3]), h*float(tokens[4])
            xmin, ymin = int(xc - w_/2), int(yc - h_/2)
            xmax, ymax = int(xc + w_/2), int(yc + h_/2)
            labels["boxes"].append([xmin, ymin, xmax, ymax])
            
    return labels


def resize_img_with_padding(img_name, size, save_path):
    xml_name = os.path.splitext(img_name)[0] + ".xml"
    txt_name = os.path.splitext(img_name)[0] + ".txt"
    if os.path.isfile(xml_name):
        labels = read_labels_from_xml(xml_name)
    elif os.path.isfile(txt_name):
        labels = read_labels_from_txt(txt_name)
    else:
        return
    
    img_name_new = os.path.join(save_path, os.path.basename(img_name))
    txt_name_new = os.path.splitext(img_name_new)[0] + ".txt"
    
    img = Image.open(img_name)
    
    # pad and crop image
    x = -((size - img.size[0]) // 2)
    y = -((size - img.size[1]) // 2)
    img_croped = img.crop(
        (
            x,
            y,
            size + x,
            size + y
        )
    )
    img_croped.save(img_name_new)

    # change label coordinates
    labels_yolo = []
    for box in labels["boxes"]:
        xmin = box[0] - x
        ymin = box[1] - y
        xmax = box[2] - x
        ymax = box[3] - y
        xcenter = (xmin+xmax)/2.0/size
        ycenter = (ymin+ymax)/2.0/size
        w = (xmax-xmin)/size
        h = (ymax-ymin)/size
        labels_yolo.append(['0', str(xcenter), str(ycenter), str(w), str(h)])
        
#         # cut cells
#         cell_name = os.path.splitext(img_name_new)[0] + '_' + str(xmin) + '_' + str(ymin) + ".jpg"
#         img_croped.crop((xmin, ymin, xmax, ymax)).save(cell_name)
        
    # write lables to txt
    with open(txt_name_new, 'w') as f:
        for label in labels_yolo:
            f.write(' '.join(label) + '\n')
            
#     print("processed ", img_name)

In [7]:
tmp_path = "../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done"
yolo_path = "../../yolo_hsil_s_20181128/hsil_s_yolo"
size = 608

image_names = [os.path.join(tmp_path, f) for f in os.listdir(tmp_path) if f.endswith(".jpg")]

for image_name in image_names:
    resize_img_with_padding(image_name, size, yolo_path)

processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-10-23-17_27_57_x27734_y43117_w112_h92.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-19-20_29_08_x25410_y7197_w146_h142.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-19-19_53_25_x14325_y26711_w142_h120.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-19-19_30_11_x37050_y37568_w96_h100.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-11-24-11_18_55_x37740_y47447_w172_h138.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-11-24-13_14_56_x12323_y19755_w116_h124.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-07-17_42_47_x12294_y19715_w184_h184.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-19-19_15_19_x46216_y29121_w128_h128.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/TC18018431_x43080_y37006_w112_h96.jpg
processed  ../../yolo_hsil_s_201

processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-07-16_53_08_x15845_y27540_w222_h170.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-19-20_29_08_x37030_y49220_w100_h106.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-19-20_06_17_x7508_y42675_w112_h132.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/TC17014226_x42919_y39399_w104_h166.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/TC18013005_x36864_y9881_w100_h122.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-07-09_21_45_x30222_y20541_w104_h174.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/TC18038024_x15196_y27857_w110_h164.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-19-18_52_06_x50338_y35678_w158_h128.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/TC18022079_x43747_y57925_w118_h98.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_do

processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-19-19_15_19_x42783_y8292_w94_h100.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/TC17035878_x34271_y31600_w106_h116.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-07-16_33_06_x29552_y31702_w94_h122.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/TC17021418_x9066_y29742_w94_h134.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-07-16_33_06_x19561_y27803_w110_h126.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-19-19_55_30_x27188_y31753_w102_h76.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-19-19_15_19_x19996_y22247_w106_h98.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-19-18_52_06_x36441_y44471_w178_h94.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/TC17011123_x28326_y39121_w114_h178.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_

processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-07-16_33_06_x19719_y43005_w162_h120.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-19-19_15_19_x31562_y34970_w122_h120.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/TC18024596_x34345_y14795_w182_h222.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-19-19_23_06_x45037_y10776_w152_h106.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-07-16_26_22_x15089_y27543_w180_h186.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-19-19_15_19_x10850_y32270_w112_h96.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-07-17_12_43_x41344_y9953_w184_h142.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-19-19_15_19_x21020_y37857_w102_h80.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/TC18023756_x15994_y34082_w134_h106.jpg
processed  ../../yolo_hsil_s_20181128/hs

processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/TC17014226_x45027_y41078_w96_h218.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-07-17_14_36_x11484_y37920_w110_h170.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-19-19_15_19_x33903_y39852_w100_h118.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-07-08_47_59_x8081_y38151_w114_h132.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/TC18051550_x13252_y13814_w138_h124.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/TC17067677_x36460_y14056_w128_h122.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/TC17026376_x19145_y43498_w116_h110.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-07-09_36_58_x30405_y8064_w126_h114.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/TC18024867_x39312_y44758_w102_h100.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-0

...

processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/TC17014226_x18562_y47628_w118_h110.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/TC18038567_x12385_y37330_w114_h122.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-19-19_15_19_x35691_y18885_w104_h114.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-19-20_38_00_x21157_y60260_w106_h182.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-19-18_54_48_x21504_y15819_w110_h152.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-19-20_34_33_x53265_y24462_w110_h178.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-11-24-12_08_21_x38330_y33482_w102_h134.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2018-04-04-17_35_51_x45289_y28082_w134_h134.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/TC18038024_x14946_y22765_w136_h168.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_a

processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/TC17035041_x15309_y42485_w138_h118.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-07-09_41_39_x36789_y24168_w102_h162.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-19-19_15_19_x16436_y47391_w104_h106.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-19-19_15_19_x38345_y19408_w82_h106.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/TC18021218_x3630_y12162_w112_h106.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-19-19_15_19_x23545_y51151_w124_h108.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/TC17011290_x28970_y34034_w126_h128.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/TC17030368_x48478_y44910_w114_h134.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-19-19_33_02_x29168_y42128_w148_h120.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_d

processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/TC17020953_x17727_y44143_w134_h106.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-19-20_31_51_x19339_y58369_w122_h108.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-19-19_30_11_x50378_y27575_w148_h138.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-19-20_44_10_x49797_y19501_w132_h152.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-07-18_08_21_x23127_y9473_w134_h134.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/TC17041936_x38039_y8488_w184_h126.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-19-19_33_02_x12662_y47328_w116_h120.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-11-24-12_55_31_x32813_y19816_w144_h120.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-19-18_52_06_x48099_y17157_w154_h106.jpg
processed  ../../yolo_hsil_s_20181128/h

processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-19-19_15_19_x40207_y49429_w132_h104.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-19-19_15_19_x25145_y28680_w108_h110.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-19-19_15_19_x13085_y19094_w92_h78.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-07-09_13_16_x34201_y11792_w114_h114.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-07-16_33_06_x27660_y21676_w138_h134.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-19-19_55_30_x38676_y34075_w108_h88.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-07-16_33_06_x29066_y36277_w132_h136.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-19-18_52_06_x31157_y24413_w166_h138.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/TC18018431_x12442_y46785_w130_h108.jpg
processed  ../../yolo_hsil_s_20

processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/TC17015494_x16245_y41343_w110_h124.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-19-19_30_11_x12669_y33570_w80_h178.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/TC17027027_x35816_y37864_w102_h120.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-10-23-14_14_36_x32979_y46283_w98_h114.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-07-16_43_24_x28895_y20974_w168_h88.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-07-16_33_06_x45935_y37251_w130_h122.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-19-19_15_19_x19155_y46181_w94_h80.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-07-16_26_22_x40714_y31658_w132_h150.jpg
processed  ../../yolo_hsil_s_20181128/hsil_s_all/HSIL_S_done/2017-09-19-18_52_06_x38984_y43935_w114_h126.jpg
processed  ../../yolo_hsil_s_20181128/hsil

### split data into train/valid

In [8]:
import random
import shutil


def split_train_valid(data_path):
    img_names = [os.path.join(data_path, f) for f in os.listdir(data_path) if f.endswith(".jpg")]

    random.shuffle(img_names)
    random.shuffle(img_names)

    split = 0.8

    train_names = img_names[:int(len(img_names)*split)]
    valid_names = img_names[int(len(img_names)*split):]

    train_path = os.path.join(data_path, "train")
    os.makedirs(train_path, exist_ok=True)
    for img_name in train_names:
        txt_name = os.path.splitext(img_name)[0] + ".txt"
        shutil.move(img_name, train_path)
        shutil.move(txt_name, train_path)

    valid_path = os.path.join(data_path, "valid")
    os.makedirs(valid_path, exist_ok=True)
    for img_name in valid_names:
        txt_name = os.path.splitext(img_name)[0] + ".txt"
        shutil.move(img_name, valid_path)
        shutil.move(txt_name, valid_path)
        

yolo_path = "../../yolo_hsil_s_20181128/hsil_s_yolo"
split_train_valid(yolo_path)

### kmeans clustering

In [9]:
def scan_files(directory, prefix=None, postfix=None):
    files_list = []
    for root, sub_dirs, files in os.walk(directory):
        for special_file in files:
            if postfix:
                if special_file.endswith(postfix):
                    files_list.append(os.path.join(root, special_file))
            elif prefix:
                if special_file.startswith(prefix):
                    files_list.append(os.path.join(root, special_file))
            else:
                files_list.append(os.path.join(root, special_file))
    return files_list

def collect_sizes_from_yolotxt(txt_name, size):
    sizes = []
    with open(txt_name, 'r') as f:
        for line in f.readlines():
            tokens = line.strip().split()
#             print(tokens)
            w = int(float(tokens[3])*size)
            h = int(float(tokens[4])*size)
            sizes.append([w, h])
    return sizes

def collect_sizes_from_yolotxt_all(txt_names, size):
    sizes = []
    for txt_name in txt_names:
        sizes += collect_sizes_from_yolotxt(txt_name, size)
    return sizes

In [10]:
yolo_path = "../../yolo_hsil_s_20181128/hsil_s_yolo"

sizes = collect_sizes_from_yolotxt_all(scan_files(yolo_path, postfix=".txt"), size)

# kmeans clustering
X = np.array(sizes)
kmeans = KMeans(n_clusters=9, random_state=2018).fit(X)
print(kmeans)

KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
    n_clusters=9, n_init=10, n_jobs=None, precompute_distances='auto',
    random_state=2018, tol=0.0001, verbose=0)


In [11]:
centers = []
for center in kmeans.cluster_centers_:
    print(center)
    centers.append(center)

[ 60.48167539 112.20418848]
[58.64543326 60.82435597]
[74.23298429 47.31849913]
[41.79749478 42.44919972]
[47.525      79.21363636]
[74.76744186 74.1744186 ]
[110.32644628  61.3553719 ]
[53.24230517 47.79043877]
[41.1141055  58.80275229]


In [12]:
tosort = {int(center[0]*center[1]):center for center in centers}
print(tosort)

{2544: array([53.24230517, 47.79043877]), 6769: array([110.32644628,  61.3553719 ]), 6786: array([ 60.48167539, 112.20418848]), 3764: array([47.525     , 79.21363636]), 2417: array([41.1141055 , 58.80275229]), 3512: array([74.23298429, 47.31849913]), 5545: array([74.76744186, 74.1744186 ]), 1774: array([41.79749478, 42.44919972]), 3567: array([58.64543326, 60.82435597])}


In [13]:
hassorted = sorted(tosort.items())
print(hassorted)
print(",  ".join(["{},{}".format(int(value[1][0]),int(value[1][1])) for value in hassorted]))

[(1774, array([41.79749478, 42.44919972])), (2417, array([41.1141055 , 58.80275229])), (2544, array([53.24230517, 47.79043877])), (3512, array([74.23298429, 47.31849913])), (3567, array([58.64543326, 60.82435597])), (3764, array([47.525     , 79.21363636])), (5545, array([74.76744186, 74.1744186 ])), (6769, array([110.32644628,  61.3553719 ])), (6786, array([ 60.48167539, 112.20418848]))]
41,42,  41,58,  53,47,  74,47,  58,60,  47,79,  74,74,  110,61,  60,112


### cut cells from yolo images/txts

In [15]:
import os
from PIL import Image

image_dir = "../../yolo_hsil_s_20181128/hsil_s_yolo/train"
save_path = "../../yolo_hsil_s_20181128/hsil_s_cell"

def cut_cell(img_name, txt_name, save_path):
    img = Image.open(img_name)
    w,h = img.size
    labels = []
    with open(txt_name, 'r') as f:
        for line in f.readlines():
            tokens = line.strip().split()
            cx, cy = float(tokens[1]), float(tokens[2])
            w_, h_ = float(tokens[3]), float(tokens[4])
            xmin, ymin = int((cx-w_/2)*w), int((cy-h_/2)*h)
            xmax, ymax = int((cx+w_/2)*w), int((cy+h_/2)*h)
            img.crop((xmin, ymin, xmax, ymax)).save(os.path.join(save_path, "{}_{}_{}_{}.jpg".format(xmin, ymin, xmax, ymax)))
    img.close()
    
img_names = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(".jpg")]
for img_name in img_names:
    txt_name = os.path.splitext(img_name)[0] + ".txt"
    cut_cell(img_name, txt_name, save_path)

### verify the integrity of images for test

In [4]:
import os
import shutil
from PIL import Image

test_path = "../../yolo_tri_20181128/test"
wrong_path = "../../yolo_tri_20181128/wrong"

img_names = [os.path.join(test_path, f) for f in os.listdir(test_path) if f.endswith(".jpg")]
wrong = 0
for img_name in img_names:
    try:
        img = Image.open(img_name)
        img.close()
    except:
        wrong += 1
        shutil.move(img_name, wrong_path)
#         print("image corrupted", img_name)
print("total: {}, wrong: {}".format(len(img_names), wrong))

total: 6670, wrong: 0


### pad test images to target sizes (608x608)
#### image at its original size, usually much smaller, is not good for direct yolov3 prediction

In [1]:
import os
from PIL import Image

def resize_img_with_padding(img_name, size, save_path):
    img_name_new = os.path.join(save_path, os.path.basename(img_name))
    txt_name_new = os.path.splitext(img_name_new)[0] + ".txt"
    
    img = Image.open(img_name)
    
    # pad and crop image
    x = -((size - img.size[0]) // 2)
    y = -((size - img.size[1]) // 2)
    img_croped = img.crop(
        (
            x,
            y,
            size + x,
            size + y
        )
    )
    img_croped.save(img_name_new)
        
    # write lables to txt
    with open(txt_name_new, 'w') as f:
        f.write("padx {} pady {}\n".format(x, y))
            
#     print("processed ", img_name)

In [2]:
test_path = "../../yolo_tri_20181128/test"
test608_path = "../../yolo_tri_20181128/test608"
size = 608

os.makedirs(test608_path, exist_ok=True)
img_names = [os.path.join(test_path, f) for f in os.listdir(test_path) if f.endswith(".jpg")]
for img_name in img_names:
    resize_img_with_padding(img_name, size, save_path=test608_path)